In [1]:
import os
from ultralytics import YOLO
from tqdm import tqdm


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\arune\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from pathlib import Path

BASE_DIR = Path("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/images")

IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".png", ".webp")

def count_images(folder_path):
    folder_path = Path(folder_path)
    counts = {}
    for subfolder in folder_path.rglob("*"):
        if subfolder.is_dir():
            # Count images in this subfolder
            img_count = len([f for f in subfolder.iterdir() if f.suffix.lower() in IMAGE_EXTENSIONS])
            if img_count > 0:
                counts[subfolder.relative_to(folder_path)] = img_count
    return counts

image_counts = count_images(BASE_DIR)

total_images = 0
for folder, count in image_counts.items():
    print(f"{folder}: {count} images")
    total_images += count

print(f"\nTotal images in dataset: {total_images}")


fixed_wing: 1011 images
hybrid: 1004 images
multi_rotor: 1031 images
single_rotor: 161 images
damage\missing_part: 731 images
damage\propeller_crack: 55 images
damage\wing_damage: 1079 images

Total images in dataset: 5072


In [33]:

import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm

IMAGE_ROOT = Path("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/Images")
LABEL_ROOT = Path("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/labels")

# Create label root folder if not exists
os.makedirs(LABEL_ROOT, exist_ok=True)



In [44]:

model = YOLO("yolov8n.pt")
print("YOLOv8 model loaded ✅")


YOLOv8 model loaded ✅


In [62]:
# Map all subfolders to class IDs
all_folders = []

for root, dirs, files in os.walk(IMAGE_ROOT):
    for d in dirs:
        rel_path = os.path.relpath(os.path.join(root, d), IMAGE_ROOT)
        if rel_path not in all_folders:
            all_folders.append(rel_path)

# Assign unique class ID
class_mapping = {folder: idx for idx, folder in enumerate(sorted(all_folders))}

print("Folder → Class ID mapping:")
for folder, class_id in class_mapping.items():
    print(f"{class_id}: {folder}")


Folder → Class ID mapping:
0: damage
1: damage\missing_part
2: damage\propeller_crack
3: damage\wing_damage
4: fixed_wing
5: hybrid
6: multi_rotor
7: single_rotor


In [ ]:
def create_yolo_label(img_path, save_path, class_id):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not read image: {img_path}")
        return False  # Skip if image unreadable

    h, w, _ = img.shape

    # Approximate bounding box covering most of the object
    x_center = 0.5
    y_center = 0.5
    width = 0.95
    height = 0.95

    with open(save_path, 'w') as f:
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    
    return True


In [ ]:
folder_counts = {}       
skipped_images = []      

for root, dirs, files in os.walk(IMAGE_ROOT):
    rel_path = os.path.relpath(root, IMAGE_ROOT)
    if rel_path == ".":
        continue  # Skip root folder

    # Label folder
    save_folder = os.path.join(LABEL_ROOT, rel_path)
    os.makedirs(save_folder, exist_ok=True)

    class_id = class_mapping[rel_path]

    img_files = [f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    success_count = 0

    for img_file in tqdm(img_files, desc=f"Labeling {rel_path}", unit="img"):
        img_path = os.path.join(root, img_file)
        save_path = os.path.join(save_folder, img_file.rsplit('.',1)[0] + '.txt')
        if create_yolo_label(img_path, save_path, class_id):
            success_count += 1
        else:
            skipped_images.append(img_path)

    folder_counts[rel_path] = success_count


Labeling damage: 0img [00:00, ?img/s]
Labeling single_rotor: 100%|██████████| 142/142 [00:01<00:00, 75.56img/s]


In [66]:
total_labeled = sum(folder_counts.values())
print("\nImage count per folder (successfully labeled):")
for folder, count in folder_counts.items():
    print(f"{folder:30}: {count}")

print(f"\nTotal successfully pre-labeled images: {total_labeled}")
print(f"Total skipped/unreadable images: {len(skipped_images)}")
if skipped_images:
    print("Example skipped images:", skipped_images[:5])



Image count per folder (successfully labeled):
damage                        : 0
damage\missing_part           : 730
damage\propeller_crack        : 55
damage\wing_damage            : 1079
fixed_wing                    : 1010
hybrid                        : 1002
multi_rotor                   : 1028
single_rotor                  : 142

Total successfully pre-labeled images: 5046
Total skipped/unreadable images: 0


In [67]:
# CSV with image path, label path, class name, bbox
all_data = []

for root, dirs, files in os.walk(LABEL_ROOT):
    for file in files:
        if file.endswith('.txt'):
            label_path = os.path.join(root, file)
            # Corresponding image path
            img_path = os.path.join(IMAGE_ROOT, os.path.relpath(root, LABEL_ROOT), file.replace('.txt', '.jpg'))
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x, y, w, h = line.strip().split()
                    class_name = [k for k,v in class_mapping.items() if v==int(class_id)][0]
                    all_data.append([img_path, label_path, class_name, class_id, x, y, w, h])

# Save CSV
with open("prelabel_metadata.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_path','label_path','class_name','class_id','x_center','y_center','width','height'])
    writer.writerows(all_data)

print("Metadata CSV saved: prelabel_metadata.csv")
print(f"Total entries in CSV: {len(all_data)}")


Metadata CSV saved: prelabel_metadata.csv
Total entries in CSV: 4947


In [73]:
import csv
import boto3

# Path to your CSV
csv_path = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/rootkey.csv"

with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    headers = next(reader)
    print("CSV Headers:", headers)

# Read CSV
with open(csv_path, 'r') as f:
    reader = csv.DictReader(f)
    creds = next(reader)  # Read the first row
    aws_access_key_id = creds[headers[0]]
    aws_secret_access_key = creds[headers[1]]

# Create S3 client
region_name = "ap-south-1"
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

print("✅ AWS credentials loaded and S3 client created")


CSV Headers: ['ï»¿Access key ID', 'Secret access key']
✅ AWS credentials loaded and S3 client created


In [74]:
# Example: List buckets
buckets = s3.list_buckets()
for b in buckets['Buckets']:
    print(b['Name'])


aerodroneimages


In [79]:
import os

BASE_DIR = os.path.abspath("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/Images")

print("BASE_DIR =", BASE_DIR)
print("Exists?", os.path.exists(BASE_DIR))


BASE_DIR = D:\Projects\Aero-AI_DroneDeliverySystem\Data\Auto_labelling\Images
Exists? True


In [81]:
import os

bucket_name = "aerodroneimages"

BASE_DIR = os.path.abspath("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/Images")

for root, dirs, files in os.walk(BASE_DIR):
    for file in files:
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            file_path = os.path.join(root, file)

            # Relative path ONLY from BASE_DIR
            rel_path = os.path.relpath(file_path, BASE_DIR)

            # Convert Windows path → S3 path
            s3_key = f"images/{rel_path.replace(os.sep, '/')}"

            s3.upload_file(file_path, bucket_name, s3_key)

print("All images uploaded to S3 correctly")


All images uploaded to S3 correctly


In [82]:
LABEL_BASE_DIR = os.path.abspath("D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling/Labels")

for root, dirs, files in os.walk(LABEL_BASE_DIR):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)

            rel_path = os.path.relpath(file_path, LABEL_BASE_DIR)
            s3_key = f"labels/{rel_path.replace(os.sep, '/')}"

            s3.upload_file(file_path, bucket_name, s3_key)

print("✅ All labels uploaded to S3 correctly")


✅ All labels uploaded to S3 correctly
